In [ ]:
!pip3 install llama-index llama-parse llama-index-embeddings-huggingface accelerate dspy-ai openpyxl langchain chromadb
!pip3 install flash-attn --no-build-isolation

!pip3 install sentencepiece protobuf

In [41]:
import json

import pandas as pd
import numpy as np
import io

import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

# from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
# from llama_index.core.embeddings import resolve_embed_model

from typing import List, Any, Callable, Optional
from pydantic import BaseModel

import gc
import load_env
import torch
from transformers import AutoModelForCausalLM
from dspy.retrieve.chromadb_rm import ChromadbRM
import chromadb
from chromadb.utils import embedding_functions
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from llama_index.readers.file import PandasExcelReader
# CHROMA_COLLECTION_NAME = "blockchain_and_ai"
# CHROMADB_DIR = "/workspace/data/db/"

from dotenv import load_dotenv
import os
load_dotenv('/workspace/repos/agentic-ai/.env')


True

In [111]:
filepath = "/workspace/data/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx"
disposition_inputs = [
  "Selling Costs",
  "Disposition Fee",
  "Net Operating Income",
  "Loan Assumption/Payoff",
  "Return of Forecasted Reserves",
  "CF Y 11",
  "Return of Maximum Offering Amount",
  "Projected Terminal Cap Rate",
  "Cash Flows"
]
dfs = pd.read_excel(filepath, sheet_name="5 - Disposition Analysis", header=None)
dfs.dropna(axis=0, how='all', inplace=True)
dfs.dropna(axis=1, how='all', inplace=True)
fee_columns = ['Disposition Fee', 'Selling Costs']
cashflow_columns = [1,2,3,4,5,6,7,8,9]
ground_truth = dfs[dfs[1].isin(disposition_inputs+cashflow_columns)].iloc[:, :2] # Get only the necessary columns
ground_truth.drop(labels=[16, 17], axis=0, inplace=True) # drop the duplicate Selling and Disposition Costs



In [135]:


def randomize_row_values(dfs: pd.DataFrame, ground_truth: pd.DataFrame) -> pd.DataFrame:
    """
    Randomly placing values in a dataframe
    """
    dfs_copy = dfs.copy()
    n_samples = np.random.randint(1,len(ground_truth),1)[0]
    testvalues = ground_truth.sample(n=n_samples)
    testidx = testvalues.index

    dfscolumns = dfs_copy.columns
    for row in testvalues.loc[testidx].iterrows():
        random_row = np.random.choice(dfs_copy.index,1)[0]
        random_col = np.random.choice(np.arange(len(dfscolumns)-1)[2:],1)[0]
        random_col1 = dfscolumns[random_col]
        random_col2 = dfscolumns[random_col+1]
        dfs_copy.loc[testidx, :2]=np.nan
        dfs_copy.loc[random_row, [random_col1, random_col2]] = row[1].values
    return dfs_copy, testvalues


def get_csv_string(dfs: pd.DataFrame) -> str:
    """
    Convert a DataFrame to a CSV formatted string
    """
    # Create a string buffer
    buffer = io.StringIO()

    # Convert the DataFrame to CSV format and write to the buffer
    dfs.to_csv(buffer, index=False)

    # Get the CSV as a string
    csv_string = buffer.getvalue()

    return csv_string



In [140]:
out = randomize_row_values(dfs, ground_truth)

/tmp/ipykernel_618/693264574.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Net Operating Income' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfs_copy.loc[random_row, [random_col1, random_col2]] = row[1].values


In [ ]:
# model_name = "EleutherAI/gpt-neo-125m"
# model_name = "clibrain/mamba-2.8b-instruct-openhermes"
# model_name = "microsoft/Phi-3-mini-128k-instruct" # 128K context window
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct" # 8K context window
# model_name = "clibrain/mamba-2.8b-instruct-openhermes" # 8K context window
print('first model load...')
model_name = "Qwen/Qwen2-1.5B-Instruct"
access_token = os.getenv('HF_TOKEN')
# model_name = "mistralai/Mistral-7B-Instruct-v0.3" # 32K context window
llm = dspy.HFModel(model=model_name, hf_device_map='auto', token=access_token)
llm.kwargs['max_new_tokens']=100
# llm.kwargs['repetition_penalty']=1.5
# llm.kwargs['do_sample']=True
# llm.kwargs['typical_p']=0.9
# llm.kwargs['temperature']=0.2


print('deleting model...')
llm.model=None
gc.collect
print('reloading model...')
llm.model=AutoModelForCausalLM.from_pretrained(model_name, quantization_config=None, 
                                               trust_remote_code=True, device_map="auto", 
                                               attn_implementation="flash_attention_2",  
                                               torch_dtype=torch.float16)

# llm.model.generation_config.pad_token_id = llm.tokenizer.eos_token_id
# llm.tokenizer.pad_token_id = llm.tokenizer.eos_token_id

filepath = "/workspace/data/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx"
docs = PandasExcelReader(sheet_name="5 - Disposition Analysis").load_data(filepath)

dspy.settings.configure(lm=llm)

!cp /workspace/repos/agentic-ai/MASTER\ -\ PYTHON\ -\ SCORING\ MODEL\ -\ MCG\ MADISON\ RIDGE\ DST\ -\ v2.0.xlsx /workspace/data

from llama_index.readers.file import PandasExcelReader
filepath = "/workspace/data/MASTER - PYTHON - SCORING MODEL - MCG MADISON RIDGE DST - v2.0.xlsx"
docs = PandasExcelReader(sheet_name="5 - Disposition Analysis", pandas_config={'keep_default_na':False}).load_data(filepath)

In [ ]:
import pydantic

class SpreadsheetValueExtractor(dspy.Signature):
    """For each of the variable name in the question extract a single value from the data. Exact variable name matches only."""

    question = dspy.InputField(desc="the question from the user will specify multiple variable names.")
    data = dspy.InputField()
    answer = dspy.OutputField(desc="only return the variable name and its value in this format: variable_name: value.")

class ExtractionCleanup(dspy.Signature):
    """Please clean up the extracted values. 
    Extract only the variables and values that are contained in the keys in the format_json."""
    format_json = dspy.InputField()
    extracted_values = dspy.InputField(desc="Contains too much information.")
    clean_list_of_variables_and_values = dspy.OutputField(desc="Only return the variables and values that are contained in the keys in the format_json.")


class OutputFormatVerification(dspy.Signature):
    # """For each variable name in the extracted values string, compare its format to the format_json and return the variable name and value if the format matches."""
    """Please assess the format of the variable names and values in the extracted values string. 
    If the format matches the variable description in format_json, return True and the variable names and values that match the format. 
    If the format does not match, return False and an empty string."""
    format_json = dspy.InputField(desc="The format of the variable names and values.")
    extracted_values = dspy.InputField(desc="The string of variable names and values extracted from the spreadsheet separated by a comma.")
    verification = dspy.OutputField(desc="Only return True or False.")
    verified_values = dspy.OutputField(desc="Output a string of variable names and values.")

# class VariableNameAndValue(pydantic.BaseModel):
#     variable_names: str
#     variable_names_description: str

# class VariableNameAndValues(pydantic.BaseModel):
#     topics: List[VariableNameAndValue]

# class FormatOutput(dspy.Signature):
#     """For each variable name in the extracted values list, compare its format to the format_json and return the variable name and value if the format matches."""
#     format_json: str = dspy.InputField(desc="The format of the variable names and values.")
#     extracted_values: VariableNameAndValues = dspy.InputField(desc="The list of variable names and values extracted from the spreadsheet.")
#     verified_values: str = dspy.OutputField(desc="The list of variable names and values that match the format_json.")
    


class SpreadSheetAnalyzer(dspy.Module):
    def __init__(self):
        super().__init__()

        self.format_validation = dspy.Predict(OutputFormatVerification)
        self.extraction_cleanup = dspy.Predict(ExtractionCleanup)
        self.generate_answer = dspy.Predict(SpreadsheetValueExtractor)
    
    def forward(self, data, question, format_json):
        predicted_value = self.generate_answer(data=data, question=question).answer
        list_of_predicted_values = self.extraction_cleanup(format_json=format_json, extracted_values=predicted_value).clean_list_of_variables_and_values
        print('---------------------------------')
        print(dspy.Prediction(list_of_predicted_values))
        # list_of_predicted_values = str(predicted_value.split('Answer: ')[-1].split(',')[:len(question.split(','))])[1:-1]
        value_verification_output = self.format_validation(format_json=format_json, extracted_values=list_of_predicted_values)
        # reason_for_format_failure = self.value_format_validation_agent(format_json=format_json, extracted_values=predicted_value).reason_for_format_failure
        print('---------------------------------')
        print('$$$$$$ predicted value:\n', predicted_value)
        print('$$$$$$ verification:\n', value_verification_output.verification)
        print('$$$$$$ validated values:\n', value_verification_output.verified_values)
        return dspy.Prediction(context=data, answer=predicted_value.answer)
        # return predicted_value, value_verification_output.verified_values, value_verification_output.verification
        # return value_verification_output
                               


In [ ]:
# class SpreadsheetValueExtractor(dspy.Signature):
#     """For each of the variable name in the question extract a single value from the data. Exact variable name matches only."""

#     question = dspy.InputField(desc="the question from the user will specify multiple variable names.")
#     data = dspy.InputField()
#     answer = dspy.OutputField(desc="only return the variable name and its value in this format: variable_name: value.")

# class ExtractionCleanup(dspy.Signature):
#     """Please clean up the extracted values. 
#     Extract only the variables and values that are contained in the keys in the format_json."""
#     format_json = dspy.InputField()
#     extracted_values = dspy.InputField(desc="Contains too much information.")
#     clean_list_of_variables_and_values = dspy.OutputField(desc="Only return the variables and values that are contained in the keys in the format_json.")

class SpreadsheetValueExtractor(dspy.Signature):
    """Extract the value for a variable names contained in the context."""

    question = dspy.InputField()
    context = dspy.InputField(desc="json string representation of a spreadsheet.")
    answer = dspy.OutputField(desc="only return the variable name and its value in this format: variable_name: value.")

class CheckValueFormat(dspy.Signature):
    """Check the format of the extracted values against its format description."""
    value = dspy.InputField(desc="String representation of the value.")
    format_description = dspy.InputField()
    verified = dspy.OutputField(desc="Only return one word, True or False.")

format_json = {'Selling Costs': 'float between 0 and 1', 'Disposition Fee': 'float between 0 and 1', 'Net Operating Income': 'float >= 0', 'Loan Assumption/Payoff': 'float <= 0', 'Return of Forecasted Reserves': 'float <= 0', 'CF Y 11': 'float >= 0', 'Return of Maximum Offering Amount': 'float <= 0', 'Projected Terminal Cap Rate': 'float between 0 and 1'}


In [ ]:
collect_dict = {k.split(': ')[0]:k.split(': ')[1] for k in collect}
verify_collect=[]
for k, v in format_json.items():
    print(k)
    format_description=v
    value = collect_dict[k]
    formatter = dspy.ChainOfThought(CheckValueFormat)
    outtest = formatter(value=value, format_description=format_description)
    verify_collect.append(outtest.verified.split("Verified: ")[-1].strip())

In [ ]:


collect=[]
for key, value in format_json.items():
    print(key)
    question=f'Extract the value for {key}. {key} is a {value}.'
    print(question)
    print()
    spreadtest = dspy.Predict(SpreadsheetValueExtractor)
    outtest = spreadtest(context=docs[0].text, question=question)
    collect.append(outtest.answer.split('Answer: ')[-1].strip())
# outtest = dspy.Prediction(spreadtest(data=docs[0].text, question=question))

In [ ]:

format_json = {'Selling Costs': 'float between 0 and 1', 'Disposition Fee': 'float between 0 and 1', 'Net Operating Income': 'float >= 0', 'Loan Assumption/Payoff': 'float <= 0', 'Return of Forecasted Reserves': 'float <= 0', 'CF Y 11': 'float >= 0', 'Return of Maximum Offering Amount': 'float <= 0', 'Projected Terminal Cap Rate': 'float between 0 and 1'}
# format_json = json.dumps(format_json)

In [ ]:
question='Selling Costs, Disposition Fee, Net Operating Income, Loan Assumption/Payoff, Return of Forecasted Reserves, CF Y 11, Return of Maximum Offering Amount, Projected Terminal Cap Rate.'
spreadsheet_agent = SpreadSheetAnalyzer()
output = spreadsheet_agent(data=docs[0].text, question=question, format_json=json.dumps(format_json))